# FeatureExtraction - Fake

Feature Extraction  - Fake 코드는

RealData 에서 선정한 FeatureSelected를 바탕으로, Fake데이터의 특징을 추출하는 코드입니다.
이렇게 추출된 데이터는 학습을 거쳐 하나의 모델로 이루어지고,

TestData로서 Real데이터가 활용되어 모델의 성능을 평가합니다

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as sp
import pywt
import os

C:\Users\SDML\anaconda3\envs\gan1\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\SDML\anaconda3\envs\gan1\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\SDML\anaconda3\envs\gan1\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


# Fake Data 합치기

가짜 데이터들의 경우, 센서별로 생성되었기 때문에, 별도의 합치는 과정이 필요합니다

<데이터 종류>

1: 5축 반경(radial axis) 방향 가속도

2: 축방향 가속도

3: 변위(displacement)

4: 전류(current)

5: 전압(voltage)

6: 가압(welding force) 방향 가속도

## 가짜데이터중 박살난 케이스 선정

In [10]:
LayerNumber = 3
NoiseDimension = 300
path_dir_NormalCurrent = './GAN_DATA/[Normal_Current]LN%d_ND%d'%(LayerNumber, NoiseDimension)

In [11]:
LayerNumber = 3
NoiseDimension = 100
path_dir_NormalVoltage = './GAN_DATA/[Normal_Vol]LN%d_ND%d'%(LayerNumber, NoiseDimension)

In [12]:
LayerNumber = 2
NoiseDimension = 100
path_dir_NormalAcc = './GAN_DATA/[Normal_Acc]LN%d_ND%d'%(LayerNumber, NoiseDimension)

In [13]:
LayerNumber = 2
NoiseDimension = 100
path_dir_MisalignCurrent = './GAN_DATA/[Misalign_Current]LN%d_ND%d'%(LayerNumber, NoiseDimension)

In [14]:
LayerNumber = 1
NoiseDimension = 300
path_dir_MisalignVoltage = './GAN_DATA/[Misalign_Vol]LN%d_ND%d'%(LayerNumber, NoiseDimension)

In [15]:
LayerNumber = 1
NoiseDimension = 200
path_dir_MisalignAcc = './GAN_DATA/[Misalign_Acc]LN%d_ND%d'%(LayerNumber, NoiseDimension)

In [16]:
# path_dir_NormalCurrent = 'D:/GANDATA/[Normal_Current]LN%d_ND%d'%(LayerNumber, NoiseDimension)
# path_dir_NormalVoltage = 'D:/GANDATA/[Normal_Vol]LN%d_ND%d'%(LayerNumber, NoiseDimension)
# path_dir_NormalAcc = 'D:/GANDATA/[Normal_Acc]LN%d_ND%d'%(LayerNumber, NoiseDimension)


In [9]:
# path_dir_MisalignCurrent = 'D:/GANDATA/[Misalign_Current]LN%d_ND%d'%(LayerNumber, NoiseDimension)
# path_dir_MisalignVoltage = 'D:/GANDATA/[Misalign_Vol]LN%d_ND%d'%(LayerNumber, NoiseDimension)
# path_dir_MisalignAcc = 'D:/GANDATA/[Misalign_Acc]LN%d_ND%d'%(LayerNumber, NoiseDimension)


In [17]:
file_list_NormalCurrent = os.listdir(path_dir_NormalCurrent)
file_list_NormalVoltage = os.listdir(path_dir_NormalVoltage)
file_list_NormalAcc = os.listdir(path_dir_NormalAcc)

In [18]:
file_list_MisalignCurrent = os.listdir(path_dir_MisalignCurrent)
file_list_MisalignVoltage = os.listdir(path_dir_MisalignVoltage)
file_list_MisalignAcc = os.listdir(path_dir_MisalignAcc)

In [19]:
def rms(x): # RMS 함수 정의
    return np.sqrt(np.mean(x**2))

In [20]:
# NoOfCondition = 1  # RPM 따로따로
NoOfData = 300
NoOfSensor = 3
NoOfFeature = 10
NoOfNormal = 300
NoOfMisalign = 300


In [21]:
for i in range(NoOfNormal):
    temp_current = pd.read_csv(path_dir_NormalCurrent+'/'+file_list_NormalCurrent[i], header= None).iloc[:,1]
    temp_voltage = pd.read_csv(path_dir_NormalVoltage+'/'+file_list_NormalVoltage[i], header= None).iloc[:,1]
    temp_acc = pd.read_csv(path_dir_NormalAcc+'/'+file_list_NormalAcc[i], header= None).iloc[:,1]
    temp_data = pd.concat([temp_current, temp_voltage,temp_acc], axis = 1,ignore_index = True)
    s1 = 'FakeNormal_%d = temp_data'%(i+1);exec(s1)
    

In [22]:
for i in range(NoOfMisalign):
    temp_current = pd.read_csv(path_dir_MisalignCurrent+'/'+file_list_MisalignCurrent[i], header= None).iloc[:,1]
    temp_voltage = pd.read_csv(path_dir_MisalignVoltage+'/'+file_list_MisalignVoltage[i], header= None).iloc[:,1]
    temp_acc = pd.read_csv(path_dir_MisalignAcc+'/'+file_list_MisalignAcc[i], header= None).iloc[:,1]
    temp_data = pd.concat([temp_current, temp_voltage,temp_acc], axis = 1,ignore_index = True)
    s1 = 'FakeMisalign_%d = temp_data'%(i+1);exec(s1)
    

# Feature Extraction

### Time Domain 특징값 추출 (10 features * 2 sensors = 20개씩)

In [23]:
FakeMisalign_1

,0,1,2
0,-0.347144,-0.215276,-0.107378
1,0.410271,-0.078224,-0.120568
2,-0.246246,-0.024114,-0.079020
3,0.094586,-0.007352,-0.109323
4,0.039772,-0.183764,-0.112202
...,...,...,...
2769,0.215470,0.000605,-0.208319
2770,0.382773,-0.032588,-0.098643
2771,-0.420667,-0.073586,-0.090130
2772,0.419656,-0.049284,-0.126697


In [24]:
# 특징데이터 크기 지정
TimeFeature_Normal   = np.zeros((NoOfSensor*NoOfFeature , NoOfData))
TimeFeature_Misalign   = np.zeros((NoOfSensor*NoOfFeature , NoOfData))

for i in range(NoOfData):
    
    s1 = 'temp_data1 = FakeNormal_%d'%(i+1) # 임시 Normal 데이터
    exec(s1)
    # Time Domain 특징값 추출
    for j in range(NoOfSensor):
        
        # Normal Time Domain Feature
        TimeFeature_Normal[10*j+0, i] = np.max(temp_data1.iloc[:,j])
        TimeFeature_Normal[10*j+1, i] = np.min(temp_data1.iloc[:,j])
        TimeFeature_Normal[10*j+2, i] = np.mean(temp_data1.iloc[:,j])
        TimeFeature_Normal[10*j+3, i] = rms(temp_data1.iloc[:,j])
        TimeFeature_Normal[10*j+4, i] = np.var(temp_data1.iloc[:,j])
        TimeFeature_Normal[10*j+5, i] = sp.skew(temp_data1.iloc[:,j])
        TimeFeature_Normal[10*j+6, i] = sp.kurtosis(temp_data1.iloc[:,j])
        TimeFeature_Normal[10*j+7, i] = np.max(temp_data1.iloc[:,j])/rms(temp_data1.iloc[:,j])
        TimeFeature_Normal[10*j+8, i] = rms(temp_data1.iloc[:,j])/np.mean(temp_data1.iloc[:,j])
        TimeFeature_Normal[10*j+9, i] = np.max(temp_data1.iloc[:,j])/np.mean(temp_data1.iloc[:,j])
        
        
print(TimeFeature_Normal.shape)

(30, 300)


In [25]:
for i in range(NoOfData):
    
    s1 = 'temp_data1 = FakeMisalign_%d'%(i+1) # 임시 Normal 데이터
    exec(s1)
    # Time Domain 특징값 추출
    for j in range(NoOfSensor):
        
        # Misalign Time Domain Feature
        TimeFeature_Misalign[10*j+0, i] = np.max(temp_data1.iloc[:,j])
        TimeFeature_Misalign[10*j+1, i] = np.min(temp_data1.iloc[:,j])
        TimeFeature_Misalign[10*j+2, i] = np.mean(temp_data1.iloc[:,j])
        TimeFeature_Misalign[10*j+3, i] = rms(temp_data1.iloc[:,j])
        TimeFeature_Misalign[10*j+4, i] = np.var(temp_data1.iloc[:,j])
        TimeFeature_Misalign[10*j+5, i] = sp.skew(temp_data1.iloc[:,j])
        TimeFeature_Misalign[10*j+6, i] = sp.kurtosis(temp_data1.iloc[:,j])
        TimeFeature_Misalign[10*j+7, i] = np.max(temp_data1.iloc[:,j])/rms(temp_data1.iloc[:,j])
        TimeFeature_Misalign[10*j+8, i] = rms(temp_data1.iloc[:,j])/np.mean(temp_data1.iloc[:,j])
        TimeFeature_Misalign[10*j+9, i] = np.max(temp_data1.iloc[:,j])/np.mean(temp_data1.iloc[:,j])
        

In [26]:
TimeFeature = np.concatenate([TimeFeature_Normal,TimeFeature_Misalign] , axis=1)
TimeFeature.shape

(30, 600)

### Frequency Domain 특징값 추출 (10 features * 8 wavelet levels * 2 sensors = 160개씩)

In [27]:
# Wavelet options
MotherWavelet = pywt.Wavelet('haar')   # Mother wavelet (모함수) 지정
Level   = 8                            # Wavelet 분해 레벨 지정
select  = 8                            # 특징추출 영역 고주파 영역부터 개수 지정 (d1~)

In [28]:
#Frequency Domain 특징값 추출 (Wavelet Transform 기반)
FreqFeature_Normal   = np.zeros(shape=(NoOfSensor*NoOfFeature*select , NoOfData))

for i in range(NoOfData):
    
    # 데이터 불러오기
    s1 = 'temp_data1 = FakeNormal_%d'%(i+1) # 임시 Normal 데이터
    exec(s1)
    Coef1      = pywt.wavedec(temp_data1, MotherWavelet, level=Level, axis=0)
    
    # Frequency Domain 특징값 추출
    for j in range(NoOfSensor):
        
        for k in np.arange(select):
            coef1 = Coef1[Level-k]
            
            # 영상과 다름 주의! : 영상에서는 (i-1)이지만 i 가 맞는 코드
            # Normal Frequency Domain Feature
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+0 , i] = np.max(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+1 , i] = np.min(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+2 , i] = np.mean(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+3 , i] = np.var(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+4 , i] = rms(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+5 , i] = sp.skew(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+6 , i] = sp.kurtosis(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+7 , i] = np.max(coef1[:,j])/rms(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+8 , i] = rms(coef1[:,j])/np.mean(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+9 , i] = np.max(coef1[:,j])/np.mean(coef1[:,j])
            
           



In [29]:
#Frequency Domain 특징값 추출 (Wavelet Transform 기반)
FreqFeature_Misalign   = np.zeros(shape=(NoOfSensor*NoOfFeature*select , NoOfData))

for i in range(NoOfData):
    
    # 데이터 불러오기
    s1 = 'temp_data1 = FakeMisalign_%d'%(i+1) # 임시 Normal 데이터
    exec(s1)
    Coef1      = pywt.wavedec(temp_data1, MotherWavelet, level=Level, axis=0)
    
    # Frequency Domain 특징값 추출
    for j in range(NoOfSensor):
        
        for k in np.arange(select):
            coef1 = Coef1[Level-k]
            
            # 영상과 다름 주의! : 영상에서는 (i-1)이지만 i 가 맞는 코드
            # Misalign Frequency Domain Feature
            FreqFeature_Misalign[NoOfFeature*j*select+k*NoOfFeature+0 , i] = np.max(coef1[:,j])
            FreqFeature_Misalign[NoOfFeature*j*select+k*NoOfFeature+1 , i] = np.min(coef1[:,j])
            FreqFeature_Misalign[NoOfFeature*j*select+k*NoOfFeature+2 , i] = np.mean(coef1[:,j])
            FreqFeature_Misalign[NoOfFeature*j*select+k*NoOfFeature+3 , i] = np.var(coef1[:,j])
            FreqFeature_Misalign[NoOfFeature*j*select+k*NoOfFeature+4 , i] = rms(coef1[:,j])
            FreqFeature_Misalign[NoOfFeature*j*select+k*NoOfFeature+5 , i] = sp.skew(coef1[:,j])
            FreqFeature_Misalign[NoOfFeature*j*select+k*NoOfFeature+6 , i] = sp.kurtosis(coef1[:,j])
            FreqFeature_Misalign[NoOfFeature*j*select+k*NoOfFeature+7 , i] = np.max(coef1[:,j])/rms(coef1[:,j])
            FreqFeature_Misalign[NoOfFeature*j*select+k*NoOfFeature+8 , i] = rms(coef1[:,j])/np.mean(coef1[:,j])
            FreqFeature_Misalign[NoOfFeature*j*select+k*NoOfFeature+9 , i] = np.max(coef1[:,j])/np.mean(coef1[:,j])
            
           



In [30]:
FreqFeature = np.concatenate([FreqFeature_Normal,FreqFeature_Misalign] , axis=1)
FreqFeature.shape

(240, 600)

# Feature 통합

In [31]:
Features = np.concatenate([TimeFeature,FreqFeature] , axis=0)
FeatureData = pd.DataFrame(Features)
print("Feature Data Size :", Features.shape)
path = './ProcessedData/[Fake]FeatureData'   # path = '파일 경로/저장할 파일 이름'
FeatureData.to_csv(path, sep=',', header=None , index=None)

Feature Data Size : (270, 600)


### .
### 
### .

In [32]:
import seaborn           as sb
import scipy.stats       as sp
import matplotlib.pyplot as plt
from scipy.stats import f_oneway


특징 데이터 로드

In [33]:
s8 = "FeatureData = pd.read_csv('./ProcessedData/[Fake]FeatureData', sep=',', header=None)"
exec(s8)
print(FeatureData.shape)

(270, 600)



# 특징데이터 정상/고장1/고장2 분리

In [34]:
Normal_FeatureData   = FeatureData.iloc[:,:NoOfData]
Misalign_FeatureData = FeatureData.iloc[:,NoOfData:]

print(Normal_FeatureData.shape)
print(Misalign_FeatureData.shape)

(270, 300)
(270, 300)


#### Train에서 정한 P-value 상위 30개 기반으로 Test Feature Selection

In [35]:
P_value_Rank = pd.read_csv('./ProcessedData/P_value_Rank', header = None)

In [36]:
P_value_Rank

,0,1,2
0,234.0,WT_Acc_d5_var,0.000000e+00
1,240.0,WT_Acc_d6_max,0.000000e+00
2,244.0,WT_Acc_d6_var,0.000000e+00
3,243.0,WT_Acc_d6_rms,0.000000e+00
4,233.0,WT_Acc_d5_rms,2.643251e-321
...,...,...,...
265,179.0,WT_Voltage_d7_shape,9.062782e-01
266,89.0,WT_Current_d6_shape,9.100781e-01
267,178.0,WT_Voltage_d7_impulse,9.642204e-01
268,218.0,WT_Acc_d3_impulse,9.652670e-01


In [37]:
Rank = 30

Normal   = np.zeros((Rank,NoOfNormal))
Misalign = np.zeros((Rank,NoOfMisalign))

for i in range(Rank):
    
    index         = int(P_value_Rank.iloc[i,0])
    Normal[i,:]   = Normal_FeatureData.iloc[index,:].values
    Misalign[i,:]   = Misalign_FeatureData.iloc[index,:].values
# 정상, 고장 특징값 합치기    
FeatureSelected = pd.DataFrame(np.concatenate([Normal, Misalign] , axis=1))

print("Selected Feature Data Size :", FeatureSelected.shape)

Selected Feature Data Size : (30, 600)


In [38]:
path = './ProcessedData/[Fake]FeatureSelected'   # path = '파일 경로//저장할 파일 이름'
FeatureSelected.to_csv(path, sep=',', header=None, index=None)